<a href="https://colab.research.google.com/github/Adeseye1907/My_Project_Work/blob/main/ANN_Checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# -----------------------------------------------------------
# I. DATA LOADING AND PREPROCESSING
# -----------------------------------------------------------
print("--- I. DATA LOADING AND PREPROCESSING ---")

# Load the dataset (Ensure 'iris.csv' is uploaded to Colab session)
try:
    df = pd.read_csv('iris.csv')
    print("1. Dataset loaded successfully.")
except FileNotFoundError:
    print("ERROR: 'iris.csv' not found. Please upload the file to your Colab session.")
    exit()

# Separate Features (X) and Target (y)
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
input_features = X.shape[1]  # Should be 4

# 1. Encode Categorical Labels (Target)
le = LabelEncoder()
y_encoded = le.fit_transform(y)
num_classes = len(le.classes_)
target_names = le.classes_
print(f"2. Target labels encoded (0, 1, 2). Number of classes: {num_classes}")

# 2. Normalize Feature Data
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)
print("3. Features normalized using StandardScaler (Mean=0, StdDev=1).")

# 3. Split the Data
X_train, X_test, y_train, y_test = train_test_split(
    X_normalized,
    y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)
print(f"4. Data split: Training samples ({len(X_train)}), Testing samples ({len(X_test)}).")

# One-Hot Encode the Target for Keras/Softmax
y_train_ohe = to_categorical(y_train, num_classes=num_classes)
y_test_ohe = to_categorical(y_test, num_classes=num_classes)
print("5. Target data converted to One-Hot Encoding for training.")
print("-" * 50)


# -----------------------------------------------------------
# II. MODEL BUILDING AND COMPILATION
# -----------------------------------------------------------
print("--- II. MODEL BUILDING AND COMPILATION ---")

# Configuration: Input Layer (4 features), Hidden Layers (relu), Output Layer (softmax)
model = Sequential([
    # Input Layer: Must match the number of input features (4 for Iris)
    Dense(units=16, activation='relu', input_shape=(input_features,)),

    # Hidden Layer (Required: at least one hidden layer with 'relu' activation)
    Dense(units=8, activation='relu'),

    # Output Layer: Must match the number of classes (3 for Iris). Use 'softmax' for multi-class.
    Dense(units=num_classes, activation='softmax')
])

# Model Compilation
model.compile(
    optimizer='adam',              # Optimizer: 'adam' for reliable performance
    loss='categorical_crossentropy', # Loss: 'categorical_crossentropy' for multi-class classification
    metrics=['accuracy']           # Metric: 'accuracy' as the primary metric
)

print("Model Architecture:")
model.summary()
print("Model Compiled with Adam Optimizer and Categorical Crossentropy loss.")
print("-" * 50)


# -----------------------------------------------------------
# III. MODEL TRAINING
# -----------------------------------------------------------
print("--- III. MODEL TRAINING ---")
# Train the model, utilizing the validation set to monitor performance.
# Adjust epochs and batch size based on validation outcomes.
history = model.fit(
    X_train, y_train_ohe,
    epochs=100,
    batch_size=32,
    validation_data=(X_test, y_test_ohe), # Monitor performance on the validation set (X_test, y_test_ohe)
    verbose=1 # Show training progress
)
print("\nModel Training Complete.")
print("-" * 50)


# -----------------------------------------------------------
# IV. MODEL EVALUATION
# -----------------------------------------------------------
print("--- IV. MODEL EVALUATION ---")

# 1. Evaluate Model Performance using the Test Set
print("1. Performance on Test Set:")
loss, accuracy = model.evaluate(X_test, y_test_ohe, verbose=0)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%\n")

# 2. Generate Predictions
# Get the probability predictions (One-Hot Encoded format)
y_pred_probs = model.predict(X_test, verbose=0)

# Convert probabilities to single class labels (0, 1, 2)
y_pred_classes = np.argmax(y_pred_probs, axis=1)

# 3. Utilize Detailed Metrics

# a. Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)
print("a. Confusion Matrix:")
print(conf_matrix)

# b. Classification Report
class_report = classification_report(y_test, y_pred_classes, target_names=target_names)
print("\nb. Classification Report:")
print(class_report)

print("-" * 50)
print("Pipeline execution complete.")


--- I. DATA LOADING AND PREPROCESSING ---
1. Dataset loaded successfully.
2. Target labels encoded (0, 1, 2). Number of classes: 3
3. Features normalized using StandardScaler (Mean=0, StdDev=1).
4. Data split: Training samples (120), Testing samples (30).
5. Target data converted to One-Hot Encoding for training.
--------------------------------------------------
--- II. MODEL BUILDING AND COMPILATION ---
Model Architecture:


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            27 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 243 (972.00 B)

 Trainable params: 243 (972.00 B)

 Non-trainable params: 0 (0.00 B)

Model Compiled with Adam Optimizer and Categorical Crossentropy loss.
--------------------------------------------------
--- III. MODEL TRAINING ---
Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - accuracy: 0.6406 - loss: 0.9282 - val_accuracy: 0.5333 - val_loss: 0.9427
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6529 - loss: 0.9157 - val_accuracy: 0.6667 - val_loss: 0.9218
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.6044 - loss: 0.8976 - val_accuracy: 0.7000 - val_loss: 0.9010
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.6085 - loss: 0.8682 - val_accuracy: 0.7333 - val_loss: 0.8812
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6500 - loss: 0.8511 - val_accuracy: 0.7667 - val_loss: 0.8615
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6762 - loss: 0.8234 - val_accuracy: 0.7667 - val_loss: 0.8421
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7088 - loss: 0.7943 - val_accur